The inference service is running. We can call the target endpoint and receive predictions for feature values in `./input.json`.

In [2]:
import json
import subprocess
import requests

In [3]:
INPUT_PATH = "./input.json"

In [28]:
def get_kubectl_output(command):
    """Execute a shell command and return the output."""
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        raise Exception(f"Command failed: {result.stderr}")
    return result.stdout.strip()

In [ ]:
try:
    # Get the ingress host
    ingress_host = get_kubectl_output("kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'")

    # Get the ingress port
    ingress_port = get_kubectl_output("kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name==\"http2\")].port}'")

    # Get the service hostname
    service_hostname = get_kubectl_output("kubectl get inferenceservice iris-classifier -o jsonpath='{.status.url}' | cut -d '/' -f 3")

    with open(INPUT_PATH, "rb") as f:
        data = f.read()

    url = f"http://{ingress_host}:{ingress_port}/v2/models/iris-classifier/infer"
    headers = {
        "Host": service_hostname,
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, data=data)
    print("\nResponse:")
    response_json = json.loads(response.text)
    prediction = response_json["outputs"][0]["data"][0]
    print("Prediction is:", prediction)
except Exception as e:
    print(f"An error occurred: {e}")